<a href="https://colab.research.google.com/github/tyrypic/python-pil-labworks/blob/main/variant2_lab3_perlik_k_4_101_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №3: «Гистограмма и преобразование интенсивностей полутоновых изображений»
**Вариант 2**

**Цель:** Освоить способы трансформации изображения попиксельной обработкой и использование гистограмм.

**Задание:**
1. Возьмите полученную в 1-й лабораторной работе картинку и переведите её в grayscale-режим  
   (Y = 0.2126·R + 0.7152·G + 0.0722·B), далее используйте это полутоновое изображение вместе с тремя другими полутоновыми картинками.
2. Постройте гистограммы всех изображений.
3. Выполните логарифмическое преобразование изображений, постройте гистограммы результатов.
4. Выполните степенные (гамма) преобразования изображений, постройте гистограммы результатов.
5. Реализуйте кусочно-линейные преобразования с двумя разными наборами параметров, постройте гистограммы.
6. Выполните пороговую сегментацию для многопиковых гистограмм, локализуйте отдельные объекты и покажите результаты.

*Исходные файлы разместите в папке `images/`, результаты будете сохранять в `output/`.*

In [2]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Создаем директорию для выходных файлов
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Пути к исходным изображениям (замените на свои файлы)
image_paths = [
    'images/lab1_input.png',    # результат первой лаб. работы
    'images/gray1.jpg',
    'images/gray2.jpg',
    'images/gray3.jpg'
]

def rgb_to_gray(image):
    """Преобразование в полутоновый канал по формуле Y = 0.2126 R + 0.7152 G + 0.0722 B."""
    arr = np.array(image).astype(float)
    y = 0.2126 * arr[...,0] + 0.7152 * arr[...,1] + 0.0722 * arr[...,2]
    gray = np.uint8(y)
    return Image.fromarray(gray)

def plot_histogram(arr, title, filename):
    """Строим и сохраняем гистограмму."""
    plt.figure()
    plt.hist(arr.ravel(), bins=256, range=(0,255))
    plt.title(title)
    plt.xlabel('Интенсивность')
    plt.ylabel('Частота')
    plt.savefig(os.path.join(output_dir, filename))
    plt.close()

def log_transform(image):
    arr = np.array(image).astype(float)
    c = 255 / np.log(1 + np.max(arr))
    log_img = c * np.log(1 + arr)
    log_img = np.uint8(np.clip(log_img, 0, 255))
    return Image.fromarray(log_img)

def power_law_transform(image, gamma):
    arr = np.array(image).astype(float) / 255.0
    pow_img = np.power(arr, gamma) * 255
    pow_img = np.uint8(np.clip(pow_img, 0, 255))
    return Image.fromarray(pow_img)

def piecewise_linear(image, r1, s1, r2, s2):
    arr = np.array(image).astype(float)
    out = np.zeros_like(arr)
    # первый участок
    mask1 = arr <= r1
    out[mask1] = (s1/r1) * arr[mask1]
    # второй
    mask2 = (arr > r1) & (arr <= r2)
    out[mask2] = ((s2 - s1)/(r2 - r1)) * (arr[mask2] - r1) + s1
    # третий
    mask3 = arr > r2
    out[mask3] = ((255 - s2)/(255 - r2)) * (arr[mask3] - r2) + s2
    out = np.uint8(np.clip(out, 0, 255))
    return Image.fromarray(out)

def threshold_segmentation(image, thresh):
    arr = np.array(image)
    seg = np.where(arr > thresh, 255, 0).astype(np.uint8)
    return Image.fromarray(seg)

# Обработка всех изображений
for path in image_paths:
    img = Image.open(path).convert('RGB')
    base = os.path.splitext(os.path.basename(path))[0]

    # 1. Полутоновое преобразование
    gray = rgb_to_gray(img)
    gray.save(os.path.join(output_dir, f'{base}_gray.png'))
    plot_histogram(np.array(gray), f'Гистограмма {base} (gray)', f'{base}_hist_gray.png')

    # 3. Лог-преобразование
    log_img = log_transform(gray)
    log_img.save(os.path.join(output_dir, f'{base}_log.png'))
    plot_histogram(np.array(log_img), f'Гистограмма {base} (log)', f'{base}_hist_log.png')

    # 4. Гамма-преобразования с разными гамма
    for g in [0.5, 1.5, 2.2]:
        pow_img = power_law_transform(gray, g)
        pow_img.save(os.path.join(output_dir, f'{base}_gamma_{g}.png'))
        plot_histogram(np.array(pow_img), f'Гистограмма {base} (gamma={g})', f'{base}_hist_gamma_{g}.png')

    # 5. Кусочно-линейные преобразования (два набора)
    pw1 = piecewise_linear(gray, r1=70, s1=0, r2=140, s2=255)
    pw1.save(os.path.join(output_dir, f'{base}_pw1.png'))
    plot_histogram(np.array(pw1), f'Гистограмма {base} (piecewise1)', f'{base}_hist_pw1.png')

    pw2 = piecewise_linear(gray, r1=50, s1=30, r2=200, s2=220)
    pw2.save(os.path.join(output_dir, f'{base}_pw2.png'))
    plot_histogram(np.array(pw2), f'Гистограмма {base} (piecewise2)', f'{base}_hist_pw2.png')

    # 6. Пороговая сегментация (выберите порог по гистограмме)
    thresh = 128
    seg = threshold_segmentation(gray, thresh)
    seg.save(os.path.join(output_dir, f'{base}_seg_{thresh}.png'))
    plot_histogram(np.array(seg), f'Гистограмма {base} (seg {thresh})', f'{base}_hist_seg_{thresh}.png')

print("Обработка завершена. Результаты сохранены в папке output.")

Обработка завершена. Результаты сохранены в папке output.
